# Design Notes

2024-11-23

Ver A2.3

In [9]:
DATE = "2024-11-23"
VERSION = "A2.3"

VCC = 5
Vio = 3.3

## General PCB ✅

- ✅ Place ground plan below ⇒ `EXT GND` and `HALL GND`
- ✅ Mounting holes (×4): 2.2 mm
- ✅ Check PCBWay standards: [Here][1]
- Signal thickness: $0.2\ \text{mm}$ ($8\ \text{mills}$)

[1]: https://www.pcbway.com/capabilities.html

## I²C Bus ✅

$$R_{\text{min}} = \frac{V_{\text{DD,max}} - V_{\text{OL,MAX}}}{I_{\text{OL}}}$$

- System IO Voltage: $V_{\text{DD}} = 3.3 \ \text{V} \pm 10 \% \Rightarrow V_{\text{DD,MAX}} = 3.36 \text{V}$
- I²C Spec: $V_{\text{OL,MAX}} = 0.4 \ \text{V}$
- I²C Spec: $I_{\text{OL}} = 3 \ \text{mA}$

In [10]:
Vddmax = 3.36
Volmax = 0.4
Iol = 3e-3
Rmin = (Vddmax - Volmax)/Iol
Rmin

986.6666666666666

$$R_{\text{min}} \approx 1.1 \ \text{k}\Omega$$

$$R_{\text{max}} \approx 1.18 \cdot \frac{t_{\text{r,max}}}{C_{\text{b}}}$$

- I²C max rise time (400kHz): $t_{\text{r,max}} = 300 \ \text{ns}$
- Max supported capa: $C_{\text{b,max}} = 400 \ pF$ ⇒ $R_{\text{max}} = 880 \ \Omega$
- Schema load capa ($10 \ \text{pF}$ per device + $2.2\ \text{pF}$ per cm of microstrip): $50\ \text{pF}$ ⇒ $R_{\text{max}} = 7 \ \text{k}\Omega$

In [11]:
trmax = 300e-9
Cbmaxth = 400e-12 # Theoretical
Rmaxth = 1.18 * (trmax / Cbmaxth)
Cbmaxr = 50e-12 # Realistic
Rmaxr = 1.18 * (trmax / Cbmaxr)
(Rmaxth, Rmaxr)

(884.9999999999998, 7079.999999999998)

<div style="border: 1px solid red;">

$$R_{\text{pu},I^2C} = 2.2 \ \text{k}\Omega$$

</div>

Route the SDA and SCL away from each other (space at least 3H), pass through the pull-up resistorsn keep the lines as short as possible, as thin as possible (increases capacitance) ⇒

<div style="border: 1px solid red;">

$$\text{microstrip thickness} = 0.2\ \text{mm} (8\ \text{mills})$$

</div>

Place over <span style="color: red">**static plane**</span> (GND or 5V).

## SPI Bus ✅

No particular restriction, not specifically high speed, no pull-ups, no serie resistor

<div style="border: 1px solid red;">

$$\text{microstrip thickness} = 0.2\ \text{mm} (8\ \text{mills})$$

</div>

## Hall sensors ✅

`A3144` was validated on direct input of the ESP32. $10\ \text{k}\Omega$ resistors were validated as functional. $5\ \text{V}$ input worked OK.

## Battery Pack ✅

- It looks like the battery pack **needs to be drained** of current every 5s, sometimes minutes.
- $22\,\Omega$ resistor is proposed, which drains $227\,\text{mA}$ ⇒ $1.14\,\text{W}$.
- This is **too heavy** for standard $1/4\ \text{W}$ resistors. ⇒
  - Resistor will be loaded for too little time (few ns)
  - Use $4 \times (4 \times 22\ \Omega) = 4 \times 100\ \Omega$
  - Use $1\ \text{W}$-rated resistor

⇒ Solution: Use 2 parallel $100\ \Omega$ resistors ✅

In [12]:
Rbatt = 22
Irbatt = VCC/Rbatt
Prbatt = VCC * Irbatt
(Irbatt, Prbatt)

(0.22727272727272727, 1.1363636363636362)

- 2N7000 Max I: $0.2\ \text{A}$, Peak: $0.8\ \text{A}$ ⇒ Mostly OK

## ~~I2S bus~~

- high speed, over ground plane
- shorter as possible

- Check resistors, it doesn't seem logical at all ⇒ ~~remove (direct connection, module is enough)~~

$$\text{microstrip thickness} = 0.2\ \text{mm} (8\ \text{mills})$$

## Button & Status LED ✅

- Create a simple multi-function push-button
- Make a multicolor (RGB?) status LED

In [13]:
Vr = 1.76 # Ir = 44nA
Vb = 2.60 # Ir = 44nA
Vg = 2.68 # Ir = 44nA
Irgbmax = 30e-3
Iiomax = 20e-3
Imax = min(Irgbmax, Iiomax)
Rr, Rb, Rg = (Vio-Vr)/Imax, (Vio-Vb)/Imax, (Vio-Vg)/Imax
(Rr, Rb, Rg)

(76.99999999999999, 34.999999999999986, 30.999999999999982)

Resistance selection:
- ~~Button: $1\ \text{k}\Omega$ to 5V, $1.5\ \text{k}\Omega$ to GND (voltage divider + pull-down)~~
- Red: $100\ \Omega$ to PIN
- Blue $68\ \Omega$ to PIN
- Green: $220\ \Omega$ to PIN

### Resistance selection issue for button

2nd resistance selection for button due to absence of $1.5\ \text{k}\Omega$ as SMD 0805

$$V_{\text{out}} = V_{\text{in}} \cdot \frac{R_2}{R_1 + R_2}$$

Selected $V_{\text{out}} = 3.3\,V$, $V_{\text{in}} = 5\,V$:

$$3.3 = 5 \cdot \frac{R_2}{R_1 + R_2} \implies \frac{R_2}{R_1} = \frac{0.66}{0.34} \approx 1.94$$

So, $R_2 \approx 1.94 \times R_1$
- $R_1 = 2\,\text{k}\Omega$, $R_2 = 3.3\,\text{k}\Omega$ ⇒  $V_{\text{out}} \approx 3.11\,V$
- $R_1 = 2.2\,\text{k}\Omega$, $R_2 = 4.7\,\text{k}\Omega$ ⇒  $V_{\text{out}} \approx 3.41\,V$ ✔️

$I_{\text{total}} = 0.724\,\text{mA}$

- Button: $2.2\ \text{k}\Omega$ to 5V, $4.7\ \text{k}\Omega$ to GND (voltage divider + pull-down)

In [14]:
R2a = 2*1.94
R2b = 2.2*1.94
Itot = 5/(2.2e3+4.7e3)
R2a, R2b, Itot

(3.88, 4.268, 0.0007246376811594203)

## PMA8403 Module ✅

- Place on the board
- Wire GND between the I2S signals
- Do footprint to keep spacing between slots


## Resistor Check-Up ✅

- `0805`: $1/8\ W$
- `1206`: $1/4\ W$ (Mandatory for $R8$ & $R9$)

In [15]:
P_R1R2 = 5**2 / 2.2e3
P_R3R10R16 = 3.3**2 / 1e3
P_R8R9 = 5**2 / 50
P_R11R12 = 3.3**2 / (1e3 + 1.5e3)
P_R17R40 = 5**2 / 10e3
powers = (P_R1R2, P_R3R10R16, P_R8R9, P_R11R12, P_R17R40)
{
    "powers": powers,
    "compatible": ["yes" if i < (1/8) else "no" for i in powers]
}

{'powers': (0.011363636363636364, 0.010889999999999999, 0.5, 0.004356, 0.0025),
 'compatible': ['yes', 'yes', 'no', 'yes', 'yes']}

- `0805` enough for all resistors except $R8$ and $R9$ (power drain keep alive)
- `1206` minimal for $R8$ and $R9$.

## Reserved IO Checks ✅

- Boot Mode strapping: GPIO0, GPIO2, **GPIO5**
- GPIO5 is used as VSPICS
- [Datasheet][3]
- [Technical Reference Manual][2]
- Both General Purpose SPI can use any GPIO ⇒ Move away frpom GPIO5

- ⚠️ And <span style="color: red">**GPIO 12**</span>!! (I forgot this one!) ⇒ `HSPI MOSI` needs to be moved to `GPIO34`. As my design is already sent to manufacture, I need to burn the eFuses!

[2]: https://www.espressif.com/sites/default/files/documentation/esp32_technical_reference_manual_en.pdf
[3]: https://www.espressif.com/sites/default/files/documentation/esp32_datasheet_en.pdf

## DAC ✅

- The lines are actually not I2S at all! Need the resistors and protect the lines from interferences.
- Used $22\ k\Omega$ and $4.7\ k\Omega$ resistors like [in the blog post][4]

[4]: https://www.az-delivery.de/en/blogs/azdelivery-blog-fur-arduino-und-raspberry-pi/internet-radio-mit-dem-esp32


## MOSFETs as GND switches ✅

- After remarks from https://www.eevblog.com/forum/projects/need-review-diy-calendar-house/msg5722997/#msg5722997
- Following https://www.electronics-tutorials.ws/transistor/tran_7.html
- Reworked all three switches

Still not OK, the voltage drop can reach $0.2\ V$. It's okay for the Hall sensors, but not if chained. ⇒ removing EXT_GND switch, adding specific logic to turn LEDs off using OE pin. ⇒ DONE.
